In [ ]:
import findspark

In [ ]:
import re

In [ ]:
findspark.init('/home/cse587/spark-2.4.0-bin-hadoop2.7')

In [ ]:
import pyspark

In [ ]:
from pyspark.sql import *
from pyspark.sql.functions import *

In [ ]:
sc = pyspark.SparkContext()

In [ ]:
spark = SparkSession \
    .builder \
    .appName("PA3") \
    .config("spark.some.config.option","some-value") \
    .getOrCreate()

In [ ]:
import pandas as pd
df = pd.read_csv("train.csv")
df_test =  pd.read_csv("test.csv")

In [ ]:
df = spark.createDataFrame(df)
df_test = spark.createDataFrame(df_test)

In [ ]:
df1 =df.withColumn('plot',lower(regexp_replace(df['plot'],"[^a-zA-Z ]","")))
df1_test =df_test.withColumn('plot',lower(regexp_replace(df_test['plot'],"[^a-zA-Z ]","")))

In [ ]:
from nltk.corpus import stopwords
import nltk 
nltk.download('stopwords')
df1.take(1)

In [ ]:
stop = stopwords.words('english')
df1.printSchema()


In [ ]:
df1 = df1.dropna()
df1_test = df1_test.dropna()

In [ ]:
df2 = df1.select('plot')
df2_test = df1_test.select('plot')

In [ ]:
rd = df2.rdd.flatMap(list)
rd_test = df2_test.rdd.flatMap(list)
data = df2.rdd.map(list)
data_test = df2_test.rdd.map(list)
data = data.map(lambda x: x[0].split())
data_test = data_test.map(lambda x: x[0].split())
def remove_stop_words(x):
    to_remove = []
    for word in x:
        if word not in stop:
            continue
        else:
            to_remove.append(word)
    for word in to_remove:
        x.remove(word)
    return x
data = data.map(remove_stop_words)
data_test = data_test.map(remove_stop_words)

In [ ]:
rd1 = rd.flatMap(lambda x: x.split())
rd1_test = rd_test.flatMap(lambda x:x.split())
rd1 = rd1.filter(lambda x: x not in stop)
rd1_test = rd1_test.filter(lambda x: x not in stop)

In [ ]:
m = rd1.map(lambda x: (x,1))
m_test = rd1_test.map(lambda x: (x,1))

In [ ]:
wordCount = m.reduceByKey(lambda a,b: a+b)

In [ ]:
common_words = wordCount.filter(lambda x: x[1] > 200)

In [ ]:
common_words.count()

In [ ]:
features = common_words.keys()

In [ ]:
from pyspark .mllib.linalg import SparseVector

In [ ]:
from collections import Counter
def remove_uncommon_words(x):
    for word in x:
        if word in features:
            continue
        else:
            x.remove(word)
    return x


In [ ]:
features= features.collect()


In [ ]:
data = data.map(remove_uncommon_words)
data_test = data_test.map(remove_uncommon_words)

In [ ]:
vocab_map = data.flatMap(lambda token: token).distinct() \
    .zipWithIndex().collectAsMap()

vocab_map_test = data_test.flatMap(lambda token: token).distinct() \
    .zipWithIndex().collectAsMap()

In [ ]:
v_map = sc.broadcast(vocab_map)
vocab_size = sc.broadcast(len(vocab_map))
tdm = data \
    .map(Counter) \
    .map(lambda counts: {v_map.value[token]: float(counts[token]) for token in counts})\
    .map(lambda index_counts: SparseVector(vocab_size.value, index_counts))


v_map_test = sc.broadcast(vocab_map_test)
vocab_size_test = sc.broadcast(len(vocab_map_test))
tdm_test = data_test \
    .map(Counter) \
    .map(lambda counts: {v_map_test.value[token]: float(counts[token]) for token in counts})\
    .map(lambda index_counts: SparseVector(vocab_size_test.value, index_counts))

In [ ]:
labels = df1.select('genre')

In [ ]:
labels= labels.rdd.flatMap(lambda x:x)

In [ ]:
sparseVectorsList = tdm.collect()
sparseVectorsList_Test = tdm_test.collect()

In [ ]:
movie_ids = df.select('movie_id')
movie_ids = movie_ids.rdd
movie_ids = movie_ids.flatMap(lambda x:x)
movie_ids = movie_ids.collect()


In [ ]:
keylist = pd.read_csv('mapping.csv')
keylist.rename(columns={'Unnamed: 0': 'i'}, inplace=True)
keylist = spark.createDataFrame(keylist)
k = keylist.select('0')
k1 = keylist.select('i')
k = k.rdd.flatMap(lambda x:x)
k1 = k1.rdd.flatMap(lambda x:x)
genre_list = k1.zip(k)

In [ ]:
genre_list.collect()

In [ ]:
import numpy as np
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.classification import SVMWithSGD, SVMModel


In [ ]:
# def mapFunc(line):
#     val = 0.0
#     vals = line.map(lambda x: (x[0], x[1]))
#     for lab in vals[0]:
#         if label == lab:
#             val = 1.0
#             break
#     return LabeledPoint(val, vals[1])

def test(data):
    ans = []
    for i in range(len(data)):
        val = 0.0
        labels = data[i][0].replace('[','').replace(']','').replace('\'','').split(',')
        for lab in labels:
            if lab == label:
                val = 1.0
                break
        pt = LabeledPoint(val, data[i][1])
        ans.append(pt)
    return ans

def makeClassifier(data):
#     lab2Data = data.map(mapFunc)
    lst = test(data)
    return SVMWithSGD.train(sc.parallelize(lst), iterations=100)
#     l2dl = lab2Data.collect()
#     return SVMWithSGD.train(sc.parallelize(l2dl), iterations=100)

In [ ]:
lab2sparse = labels.zip(tdm)
lab2sparse_test = labels_test.zip(tdm_test)

In [ ]:
lab2sparse_test.take(1)

In [ ]:
l2sm = lab2sparse.map(lambda x: (x[0],x[1]))
data = l2sm.collect()
l2sm_test =lab2sparse_test.map(lambda x: (x[0],x[1]))

In [ ]:
data_test = l2sm_test.collect()

In [ ]:
classifiers = []

for label in k.collect():
    print('Trained', label, 'Classifier')
    classifiers.append(makeClassifier(data))
# classifier = makeClassifier(lab2sparse)

# Part 2: TF-IDF

In [ ]:
 from pyspark.mllib.feature import HashingTF, IDF